# Classificação com Árvores de Decisão no Scikit-Learn
- Este notebook faz parte da atividades teóricas do clube de Inteligência Artificial da Escola Sesc de Ensino Médio;
- Nesta atividade iremos aprender mais sobre algoritmos de Classificação utilizando árvores de Decisão em uma biblioteca muito utilizada para Machine Learning, a Scikit-Learn.

## Scikit-Learn
- A scikit-learn (originalmente scikits.learn) é uma biblioteca de aprendizado de máquina de código aberto para a linguagem de programação Python. Ela inclui vários algoritmos de classificação, regressão e agrupamento incluindo máquinas de vetores de suporte, florestas aleatórias, gradient boosting, k-means e DBSCAN, e é projetada para interagir com as bibliotecas Python numéricas e científicas NumPy e SciPy.

https://scikit-learn.org/

## 1. Preparando os dados

### 1.1 Importando as bibliotecas principais

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

**NumPy** é um pacote para a linguagem Python que suporta arrays e matrizes multidimensionais, possuindo uma larga coleção de funções matemáticas para trabalhar com estas estruturas. *Wikipédia*

**Pandas** é uma biblioteca de software criada para a linguagem Python para manipulação e análise de dados. Em particular, oferece estruturas e operações para manipular tabelas numéricas e séries temporais. É software livre sob a licensa licença BSD. *Wikipédia*

**Matplotlib** é uma biblioteca de software para criação de gráficos e visualizações de dados em geral, feita para a da linguagem de programação Python e sua extensão de matemática NumPy. Originalmente criada pelo biólogo e neurocientista americano John D. *Wikipédia*

### 1.2 Importando  e visualizando o Dataset
**Dataset** é uma coleção de dados (conjunto de dados) normalmente tabulados. Por cada elemento se indicam várias características. Cada coluna representa uma variável particular. Cada linha corresponde a um determinado membro do conjunto de dados em questão. Cada valor é conhecido como um dado.

#### Sobre o Dataset deste projeto

**Diagnóstico de COVID-19 e seu espectro clínico**: https://www.kaggle.com/einsteindata4u/covid19

Este conjunto de dados contém dados anônimos de pacientes atendidos no Hospital Israelita Albert Einstein, em São Paulo, Brasil, e que tiveram amostras coletadas para a realização do SARS-CoV-2 RT-PCR e exames laboratoriais adicionais durante uma visita ao hospital.

Todos os dados foram tornados anônimos seguindo as melhores práticas e recomendações internacionais. Todos os dados clínicos foram padronizados para ter uma média de zero e um desvio padrão da unidade.

**Objetivo:** Com base nos resultados dos testes laboratoriais comumente coletados para um caso suspeito de COVID-19 durante uma visita ao pronto-socorro, o modelo deverá prever a classificação do resultado do teste para SARS-Cov-2 como positivo ou negativo.

**Link para o arquivo .csv do Dataset:** https://raw.githubusercontent.com/diogocortiz/Crash-Course-IA/master/ArvoreDecis%C3%A3o/dataset_einstein.csv

#### Lendo o arquivo que contém os nossos dados e criando um Dataframe com o Pandas

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/diogocortiz/Crash-Course-IA/master/ArvoreDecis%C3%A3o/dataset_einstein.csv', sep=";")

#### Mostrando as 5 primeiras linhas do Dataframe

In [ ]:
df.head(5)

#### Mostrando as 5 últimas linhas do Dataframe

In [ ]:
df.tail(5)

#### Exibindo informações importantes do Dataframe

In [ ]:
df.describe().round(2) # Resume e descreve algumas operações gerais do Dataset. Round(2) arredonda para 2 casas decimais.

In [ ]:
print(f'Tamanho do Dataframe: {df.shape[0]} linhas e {df.shape[1]} colunas') # Descreve o tamanho do dataframe em uma tupla

In [ ]:
df.info() # Descreve informaçoes de tamanho e de tipo das características

#### Removendo os valores em branco (NaN)
Removemos esses valores para evitar ruídos e distorções em nosso modelo

In [ ]:
print(f'Dataset com valores em branco: {df.shape[0]}')
print(f'Dataset sem valores em branco: {df.dropna().shape[0]}')

In [ ]:
df = df.dropna()

#### O nosso dataset está balanceado ou desbalanecado?

In [ ]:
print ('Total de registros negativos: ', df[df['SARS-Cov-2 exam result'] =='negative'].shape[0])
print ('Total de registros positivos: ', df[df['SARS-Cov-2 exam result'] =='positive'].shape[0])

### 1.3 Selecionando as características principais para o estudo

**Qual coluna utilizaremos para realizar nosso experimento?**
Quais serão variáveis independentes?
Qual será a variável dependente?

In [ ]:
df.head(2)

#### Selecionando as etiquetas (y)

In [ ]:
y = df['SARS-Cov-2 exam result'].values 
y[:5]

#### Selecionando as características (X)

In [ ]:
X = df[['Hemoglobin', 'Leukocytes', 'Basophils','Proteina C reativa mg/dL']].values
X[:5]

### 1.4 Criando os datasets de treino e de teste
Usando Sklearn para escalar os dados.
#### Importando o train_test_split da biblioteca scikit-learn
A função train_test_split nos ajuda a dividir o dataset em treino e teste com base nos parâmetros definidos

In [ ]:
from sklearn.model_selection import train_test_split

#### Utilizando a função train_test_split e inserindo os dados nas variáveis indicadas
O test_size representa a proporção do dataset que será incluída na divisão de teste. Os valores pode ir de 0.0 a 1.0.

O random_state garante que as divisões geradas sejam reproduzíveis. O Scikit-learn usa permutações aleatórias para gerar as divisões. O estado aleatório que você fornece é usado como uma semente (random seed) para o gerador de números aleatórios. Isso garante que os números aleatórios sejam gerados na mesma ordem.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

## 2. Escolher um modelo

<img src="https://scikit-learn.org/stable/_static/ml_map.png"/>

### 2.1 Árvores de Decisão
Árvores de decisão (DTs) são um método de aprendizado supervisionado não paramétrico usado para classificação e regressão. O objetivo é criar um modelo que preveja o valor de uma variável de destino, aprendendo regras de decisão simples inferidas dos recursos de dados.

<img src="https://scikit-learn.org/stable/_images/sphx_glr_plot_iris_dtc_0021.png"/>

Fontes: 
- https://scikit-learn.org/stable/modules/tree.htmls

#### Importando o algoritmo DecisionTreeClassifier de classificação da biblioteca scikit-learn

In [ ]:
from sklearn.tree import DecisionTreeClassifier, export_text
from sklearn import tree

## 3. Ajustar os dados ao modelo e fazer a classificação

### 3.1 Instanciar a classe DecisionTreeClassifier()

<img src="http://jcsites.juniata.edu/faculty/rhodes/ida/images/weatherdectree.GIF">

In [ ]:
modelo = DecisionTreeClassifier(criterion='entropy', max_depth=5)

In [ ]:
modelo.get_params()

### 3.2 Utilizando o método .fit()
O método .fit() do objeto modelo criado da classe DecisionTreeClassifier() treinará o nosso modelo e possibilitará a classificação.

In [ ]:
modelo.fit(X_train, y_train)

### 3.3 Exibindo as principais características e imprimindo a árvore de decisão

In [ ]:
import graphviz
print (modelo.feature_importances_)
nome_features = ['Hemoglobin', 'Leukocytes', 'Basophils','Proteina C reativa mg/dL']
nome_classes = modelo.classes_

fig, ax = plt.subplots(figsize=(15, 15))
tree.plot_tree(modelo,
               feature_names=nome_features,
               class_names=nome_classes,
               filled=True,
               max_depth=4,
               fontsize=10)
plt.show()

#### Observando a estrutura em texto

In [ ]:
r = export_text(modelo, feature_names=nome_features)
print(r)

### 3.4 Fazendo as predições no dataset de teste

In [ ]:
y_preds = modelo.predict(X_test)

In [ ]:
X_test[:5]

In [ ]:
y_preds[:5]

## 4. Avaliando o modelo


#### Quais foram as características de maior importância no modelo?

In [ ]:
importances = modelo.feature_importances_
indices = np.argsort(importances)[::-1]
print("Feature ranking:")

for f in range(X.shape[1]):
    print("%d. feature %d (%f)" % (f + 1, indices[f], importances[indices[f]]))
f, ax = plt.subplots(figsize=(11, 9))
plt.title("Feature ranking", fontsize = 20)
plt.bar(range(X.shape[1]), importances[indices],
    color="b", 
    align="center")
plt.xticks(range(X.shape[1]), indices)
plt.xlim([-1, X.shape[1]])
plt.ylabel("importance", fontsize = 18)
plt.xlabel("index of the feature", fontsize = 18)
plt.show()

#Indice das features
# 0 - 'Hemoglobin', 
# 1 - 'Leukocytes'
# 2 - 'Basophils',
# 3 - 'Proteina C reativa mg/dL']

#### Qual a acurácia da nossa árvore?

In [ ]:
from sklearn.metrics import accuracy_score, classification_report
print("ACURÁCIA DA ÁRVORE: ", accuracy_score(y_test, y_preds))